In [1]:
import numpy as np

In [2]:
from pandas import Series, DataFrame

In [3]:
import pandas as pd

In [4]:
from pandas import read_html

In [5]:
url_qb_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=QB&yr=2017&wk=all&rules=1'
url_rb_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=RB&yr=2017&wk=all&rules=1'
url_wb_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=WR&yr=2017&wk=all&rules=1'
url_te_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=WR&yr=2017&wk=all&rules=1'
url_def_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=DST&yr=2017&wk=all&rules=1'

In [6]:
dframe_qb = pd.io.html.read_html(url_qb_season17)
dframe_rb = pd.io.html.read_html(url_rb_season17)
dframe_wb = pd.io.html.read_html(url_wb_season17)
dframe_te = pd.io.html.read_html(url_te_season17)
dframe_def = pd.io.html.read_html(url_def_season17)

In [7]:
qb = dframe_qb[0]

In [8]:
qb['Passing']['Att']

0     553
1     581
2     440
3     505
4     492
5     565
6     540
7     575
8     561
9     477
10    536
11    490
12    523
13    481
14    529
15    496
16    515
17    420
18    442
19    397
20    453
21    571
22    469
23    549
24    204
25    476
26    429
27    238
28    349
29    330
     ... 
43     97
44     75
45     43
46    112
47     43
48     45
49     43
50     36
51     16
52     49
53     28
54     35
55     23
56     37
57     42
58      7
59     23
60     18
61      3
62     14
63      2
64      1
65      0
66      0
67      7
68      8
69      8
70      1
71      5
72      2
Name: Att, Length: 73, dtype: int64

In [14]:
#Get name from Player column and save to nameIndex
nameIndex = []
player = qb['\xa0'].get("Player")
for i in player:
    nameIndex.append(i)
    
    
nameIndex

['Russell WilsonR.\xa0Wilson',
 'Tom BradyT.\xa0Brady',
 'Carson WentzC.\xa0Wentz',
 'Alex SmithA.\xa0Smith',
 'Cam NewtonC.\xa0Newton',
 'Matthew StaffordM.\xa0Stafford',
 'Kirk CousinsK.\xa0Cousins',
 'Philip RiversP.\xa0Rivers',
 'Ben RoethlisbergerB.\xa0Roethlisberger',
 'Jared GoffJ.\xa0Goff',
 'Drew BreesD.\xa0Brees',
 'Dak PrescottD.\xa0Prescott',
 'Blake BortlesB.\xa0Bortles',
 'Case KeenumC.\xa0Keenum',
 'Matt RyanM.\xa0Ryan',
 'Andy DaltonA.\xa0Dalton',
 'Derek CarrD.\xa0Carr',
 'Tyrod TaylorT.\xa0Taylor',
 'Jameis WinstonJ.\xa0Winston',
 'Josh McCownJ.\xa0McCown',
 'Marcus MariotaM.\xa0Mariota',
 'Eli ManningE.\xa0Manning',
 'Jacoby BrissettJ.\xa0Brissett',
 'Joe FlaccoJ.\xa0Flacco',
 'Deshaun WatsonD.\xa0Watson',
 'DeShone KizerD.\xa0Kizer',
 'Jay CutlerJ.\xa0Cutler',
 'Aaron RodgersA.\xa0Rodgers',
 'Trevor SiemianT.\xa0Siemian',
 'Mitchell TrubiskyM.\xa0Trubisky',
 'Brett HundleyB.\xa0Hundley',
 'Carson PalmerC.\xa0Palmer',
 'Jimmy GaroppoloJ.\xa0Garoppolo',
 'C.J. Beathar

In [19]:
#Get Passing attempt from Att Data 

passingAtt = []
player = qb['Passing'].get("Att")
for i in player:
    passingAtt.append(i)
    

dataStore = []
dataStore.append(nameIndex)
dataStore.append(passingAtt)


In [20]:

#create a tuple for multi-indexed Dataframe
tup = [('Bye', 'Bye'),
       ('Pts', 'Pts'),
       ('Passing', 'Att'),
       ('Passing', 'Cmp'),
       ('Passing', 'Yds'),
       ('Passing', 'TD'),
       ('Passing', 'Int'),
       ('Passing', '2Pt'),
       ('Rushing', 'Att'),
       ('Rushing', 'Yds'),
       ('Rushing', 'TD'),
       ('Rushing', '2Pt'),
       ('Receiving', 'Rec'),
       ('Receiving', 'Yds'),
       ('Receiving', 'TD'),
       ('Receiving', '2Pt'),
       ('Fumbles', 'FL'),
       ('Fumbles', 'TD')
      ]



In [21]:
index = pd.MultiIndex.from_tuples(tup)

In [23]:
dframe1 = DataFrame(columns = index, index = qb.index)

In [26]:
dframe1.reindex(nameIndex)
dframe1 = dframe1.reindex(nameIndex)
dframe1




Bye  Pts Passing                      \
                                     Bye  Pts     Att  Cmp  Yds   TD  Int   
Russell WilsonR. Wilson              NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Tom BradyT. Brady                    NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Carson WentzC. Wentz                 NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Alex SmithA. Smith                   NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Cam NewtonC. Newton                  NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Matthew StaffordM. Stafford          NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Kirk CousinsK. Cousins               NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Philip RiversP. Rivers               NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Ben RoethlisbergerB. Roethlisberger  NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Jared GoffJ. Goff                    NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Drew BreesD. Brees                   NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Dak PrescottD. Prescott              NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Blake BortlesB. Bortles              NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Case KeenumC. Keenum                 NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Matt RyanM. Ryan                     NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Andy DaltonA. Dalton                 NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Derek CarrD. Carr                    NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Tyrod TaylorT. Taylor                NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Jameis WinstonJ. Winston             NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Josh McCownJ. McCown                 NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Marcus MariotaM. Mariota             NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Eli ManningE. Manning                NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Jacoby BrissettJ. Brissett           NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Joe FlaccoJ. Flacco                  NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Deshaun WatsonD. Watson              NaN  NaN     NaN  NaN  NaN  NaN  NaN   
DeShone KizerD. Kizer                NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Jay CutlerJ. Cutler                  NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Aaron RodgersA. Rodgers              NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Trevor SiemianT. Siemian             NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Mitchell TrubiskyM. Trubisky         NaN  NaN     NaN  NaN  NaN  NaN  NaN   
...                                  ...  ...     ...  ...  ...  ...  ...   
T.J. YatesT. Yates                   NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Kevin HoganK. Hogan                  NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Sam BradfordS. Bradford              NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Bryce PettyB. Petty                  NaN  NaN     NaN  NaN  NaN  NaN  NaN   
David FalesD. Fales                  NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Paxton LynchP. Lynch                 NaN  NaN     NaN  NaN  NaN  NaN  NaN   
E.J. ManuelE. Manuel                 NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Geno SmithG. Smith                   NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Ryan MallettR. Mallett               NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Nathan PetermanN. Peterman           NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Landry JonesL. Jones                 NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Patrick MahomesP. Mahomes            NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Nate SudfeldN. Sudfeld               NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Sean MannionS. Mannion               NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Matt CasselM. Cassel                 NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Joe WebbJ. Webb                      NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Cody KesslerC. Kessler               NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Scott TolzienS. Tolzien              NaN  NaN     NaN  NaN  NaN  NaN  NaN   
Cooper RushC. Rush                   NaN  NaN     NaN  NaN  NaN  NaN  NaN   
A.J. McCarronA. McCarron         